<a href="https://colab.research.google.com/github/redom13/ML/blob/main/fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Dependencies

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words('english')) #The words that are mainly used to make up a sentence
# These words don't hold much importance

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Preprocessing

In [3]:
news_dataset=pd.read_csv('/content/train.csv',index_col='id')

In [10]:
news_dataset[:5]

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
news_dataset.shape

(20800, 4)

In [14]:
news_dataset.isnull().sum()

,0
title,558
author,1957
text,39
label,0


In [4]:
news_dataset=news_dataset.fillna('')

In [5]:
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [8]:
news_dataset.head()

,title,author,text,label,content
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [6]:
y=news_dataset['label']
X=news_dataset.drop(columns='label',axis=1)

In [19]:
print(X),print(y)

                                                   title  \
id                                                         
0      House Dem Aide: We Didn’t Even See Comey’s Let...   
1      FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2                      Why the Truth Might Get You Fired   
3      15 Civilians Killed In Single US Airstrike Hav...   
4      Iranian woman jailed for fictional unpublished...   
...                                                  ...   
20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799                          What Keeps the F-35 Alive   

                                          author  \
id                                                 
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2          

(None, None)

**Stemming**: The process of reduceing a word to its root word

In [7]:
port_stem=PorterStemmer()

In [8]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content) # Substitute everyting except alphabets with space in the string-->content
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split() # Splitted the whole sentences & Converted to a list of words
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  # Stem all the words that r not stopword
  stemmed_content=' '.join(stemmed_content) # Make the sentences again(No list again)
  return stemmed_content


In [9]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [10]:
news_dataset['content']

,content
id,
0,darrel lucu hous dem aid even see comey letter...
1,daniel j flynn flynn hillari clinton big woman...
2,consortiumnew com truth might get fire
3,jessica purkiss civilian kill singl us airstri...
4,howard portnoy iranian woman jail fiction unpu...
...,...
20795,jerom hudson rapper trump poster child white s...
20796,benjamin hoffman n f l playoff schedul matchup...
20797,michael j de la merc rachel abram maci said re...


In [11]:
X=news_dataset['content'].values
y=news_dataset['label'].values

In [29]:
X,y

(array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
        'daniel j flynn flynn hillari clinton big woman campu breitbart',
        'consortiumnew com truth might get fire', ...,
        'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
        'alex ansari nato russia hold parallel exercis balkan',
        'david swanson keep f aliv'], dtype=object),
 array([1, 0, 1, ..., 0, 1, 1]))

In [12]:
vectorizer=TfidfVectorizer(max_features=9000)
vectorizer.fit(X)
X=vectorizer.transform(X)

In [13]:
print(X)

  (0, 8330)	0.28485063562728646
  (0, 7172)	0.2565896679337957
  (0, 4655)	0.3635963806326075
  (0, 4481)	0.29212514087043684
  (0, 4080)	0.24785219520671603
  (0, 3706)	0.21874169089359144
  (0, 2726)	0.233316966909351
  (0, 2047)	0.2705332480845492
  (0, 1928)	0.3598939188262559
  (0, 1580)	0.2468450128533713
  (0, 1294)	0.3676519686797209
  (0, 145)	0.27010124977708766
  (1, 8846)	0.30071745655510157
  (1, 3605)	0.1904660198296849
  (1, 3014)	0.7143299355715573
  (1, 1910)	0.26373768806048464
  (1, 1494)	0.19094574062359204
  (1, 1165)	0.3827320386859759
  (1, 1006)	0.15521974226349364
  (1, 797)	0.2939891562094648
  (2, 8286)	0.41544962664721613
  (2, 5052)	0.49351492943649944
  (2, 3252)	0.3474613386728292
  (2, 2950)	0.3866530551182615
  (2, 1662)	0.46097489583229645
  :	:
  (20797, 6982)	0.2482526352197606
  (20797, 6540)	0.27263457663336677
  (20797, 6440)	0.24778257724396507
  (20797, 5437)	0.08038079000566466
  (20797, 5033)	0.174553480255222
  (20797, 4996)	0.295420400342031

# Splitting the training set

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

# Training the Model

In [15]:
model=LogisticRegression()

In [16]:
model.fit(X_train,y_train)

LogisticRegression()

# Predicting

In [17]:
prediction_score=model.score(X_test,y_test)

In [18]:
prediction_score

0.9790865384615385

# Transform and prepare raw data for **validation dataset**

In [19]:
validation_df=pd.read_csv('/content/test.csv',index_col='id')
validation_df.shape

(5200, 3)

In [20]:
validation_df.isnull().sum()

,0
title,122
author,503
text,7


In [21]:
validation_df=validation_df.fillna('')

In [22]:
validation_df['content']=validation_df['author']+" "+validation_df['title']

In [23]:
validation_df['content']=validation_df['content'].apply(stemming)

In [24]:
X_valid=validation_df['content'].values

In [25]:
vectorizer=TfidfVectorizer(max_features=9000)
vectorizer.fit(X_valid)
X_valid=vectorizer.transform(X_valid)

In [26]:
y_valid=model.predict(X_valid)

In [27]:
y_valid.shape

(5200,)

In [29]:
submit_df=pd.read_csv('/content/submit.csv')
final_submit=pd.DataFrame()
final_submit['id']=submit_df['id']
final_submit['label']=y_valid
final_submit.to_csv('final_submit.csv',index=False)
final_submit.head()

,id,label
0,20800,1
1,20801,1
2,20802,1
3,20803,1
4,20804,1


In [30]:
final_submit

,id,label
0,20800,1
1,20801,1
2,20802,1
3,20803,1
4,20804,1
...,...,...
5195,25995,1
5196,25996,1
5197,25997,1
5198,25998,1
